In [1]:
import findspark
findspark.init()

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Spotify_Twitter")
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = (spark.read
            .option("header","true")
            .option("inferSchema","true")
            .csv("Main_Artist_DF.csv")            
            .distinct())

In [35]:
df_one_republic = (spark.read
            .option("header","true")
            .option("inferSchema","true")
            .json("one_republic.json")            
            .distinct())

In [30]:
df_harry_styles = (spark.read
            .option("header","true")
            .option("inferSchema","true")
            .json("harry_styles.json")            
            .distinct())

In [31]:
df_john_legend = (spark.read
            .option("header","true")
            .option("inferSchema","true")
            .json("john_legend.json")            
            .distinct())

In [32]:
df_bad_bunny = (spark.read
            .option("header","true")
            .option("inferSchema","true")
            .json("bad_bunny.json")            
            .distinct())

In [33]:
df_the_weekend = (spark.read
            .option("header","true")
            .option("inferSchema","true")
            .json("the_weekend.json")            
            .distinct())

In [5]:
df.head()

Row(images='https://i.scdn.co/image/ab6761610000e5eb77bf00f67e21f514dc44c485', followers=14400245, popularity=85, name='OneRepublic', genre='piano rock', artist_id='5Pwc4xIPtQLFEnJriah9YJ')

In [6]:
df.limit(5).toPandas()

,images,followers,popularity,name,genre,artist_id
0,https://i.scdn.co/image/ab6761610000e5eb77bf00f67e21f514dc44c485,14400245,85,OneRepublic,piano rock,5Pwc4xIPtQLFEnJriah9YJ
1,https://i.scdn.co/image/ab6761610000e5ebf7db7c8ede90a019c54590bb,23987672,90,Harry Styles,pop,6KImCVD70vtIoJWnq6nGn3
2,https://i.scdn.co/image/ab6761610000e5ebb5f9e28219c169fd4b9e8379,51405766,94,The Weeknd,canadian contemporary r&b,1Xyo4u8uXC1ZmMpatF05PJ
3,https://i.scdn.co/image/ab6761610000e5ebaed3c717bf1753ab928ea88d,6291069,77,John Legend,neo mellow,5y2Xq6xcjJb2jVM54GHK3t
4,https://i.scdn.co/image/ab6761610000e5eb8ee9a6f54dcbd4bc95126b14,57945768,99,Bad Bunny,reggaeton,4q3ewBCX7sLwd24euuV69X


In [68]:
df_the_weekend_summary

558

In [44]:
df_one_republic_summary = df_one_republic.groupBy("tweet_count").sum().collect()[0][0]
df_harry_styles_summary = df_harry_styles.groupBy("tweet_count").sum().collect()[0][0]
df_bad_bunny_summary = df_bad_bunny.groupBy("tweet_count").sum().collect()[0][0]
df_john_legend_summary = df_john_legend.groupBy("tweet_count").sum().collect()[0][0]
df_the_weekend_summary = df_the_weekend.groupBy("tweet_count").sum().collect()[0][0]

In [71]:
df_the_weekend_summary = df_the_weekend.groupBy("tweet_count").sum()

In [73]:
df_the_weekend_summary.show()

+-----------+----------------+
|tweet_count|sum(tweet_count)|
+-----------+----------------+
|        558|             558|
|        736|             736|
|        326|             326|
|        209|             209|
|       1321|            1321|
|        836|             836|
|        197|             197|
|        903|             903|
|        441|             441|
|       1237|            1237|
|        467|             467|
|        528|             528|
|        451|             451|
|        333|             333|
|        339|             339|
|        641|             641|
|        205|             205|
|        249|             249|
|        482|             482|
|        182|             182|
+-----------+----------------+
only showing top 20 rows



In [64]:
from pyspark.sql.functions import when, lit

df_joined = df.withColumn("tweets_last_month",
              when((df.name  == "OneRepublic"),lit(df_one_republic_summary)).
              when((df.name  == "Harry Styles"),lit(df_harry_styles_summary)).
              when((df.name  == "John Legend"),lit(df_john_legend_summary)).
              when((df.name  == "Bad Bunny"),lit(df_bad_bunny_summary)).
              when((df.name  == "The Weeknd"),lit(df_the_weekend_summary)))
            


In [65]:
df_joined.limit(5).toPandas()

,images,followers,popularity,name,genre,artist_id,tweets_last_month
0,https://i.scdn.co/image/ab6761610000e5eb77bf00f67e21f514dc44c485,14400245,85,OneRepublic,piano rock,5Pwc4xIPtQLFEnJriah9YJ,693
1,https://i.scdn.co/image/ab6761610000e5ebf7db7c8ede90a019c54590bb,23987672,90,Harry Styles,pop,6KImCVD70vtIoJWnq6nGn3,2520
2,https://i.scdn.co/image/ab6761610000e5ebb5f9e28219c169fd4b9e8379,51405766,94,The Weeknd,canadian contemporary r&b,1Xyo4u8uXC1ZmMpatF05PJ,558
3,https://i.scdn.co/image/ab6761610000e5ebaed3c717bf1753ab928ea88d,6291069,77,John Legend,neo mellow,5y2Xq6xcjJb2jVM54GHK3t,4139
4,https://i.scdn.co/image/ab6761610000e5eb8ee9a6f54dcbd4bc95126b14,57945768,99,Bad Bunny,reggaeton,4q3ewBCX7sLwd24euuV69X,1463


https://www.geeksforgeeks.org/how-to-add-a-new-column-to-a-pyspark-dataframe/